# Crisis Buying - Bear Market Opportunity Scanner

**Strategy:**
- ONLY activate during market corrections/bear markets
- Buy quality stocks at panic-driven discounts
- VIX > 25 = Opportunity (not danger)
- Calculate intrinsic value vs current price
- Dollar-cost average into positions

**Philosophy:**
> "Be fearful when others are greedy, and greedy when others are fearful" - Warren Buffett

This is NOT for normal trading - this is for building generational wealth during crashes.

In [1]:
import sys
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from data_fetcher import StockDataFetcher
from indicators import sma, ema, rsi, adx, atr, cmf
from market.regime import check_market_health

plt.style.use('seaborn-v0_8-darkgrid')

Extended indicators module loaded


## Market Regime Check - Crisis Mode

This scanner ONLY works during corrections/bear markets

In [2]:
# Check if we're in crisis conditions
market_regime = check_market_health(verbose=True, check_breadth=False)

print("\n" + "="*60)
print("CRISIS BUYING MODE")
print("="*60)

if market_regime['regime'] in ['bear', 'correction']:
    print("STATUS: ACTIVE - Crisis conditions detected")
    print("This is when generational wealth is built.")
    CRISIS_MODE_ACTIVE = True
elif market_regime['confidence'] < 60:
    print("STATUS: WATCH - Market showing weakness")
    print("Prepare watchlist, don't deploy capital yet.")
    CRISIS_MODE_ACTIVE = False
else:
    print("STATUS: INACTIVE - Market is healthy")
    print("Use 02.1_price_finder.ipynb for normal trading.")
    CRISIS_MODE_ACTIVE = False

print("="*60)

/Users/jaspervalk/Documents/market-analysis/notebooks/../src/market/regime.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, progress=False)
/Users/jaspervalk/Documents/market-analysis/notebooks/../src/market/regime.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, progress=False)
/Users/jaspervalk/Documents/market-analysis/notebooks/../src/market/regime.py:36: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period=period, progress=False)


MARKET REGIME ANALYSIS

Regime: NEUTRAL
Confidence: 67%
Recommendation: BUY signals valid with caution

Details:
  [HEALTHY] SPY mixed signals
  [HEALTHY] QQQ mixed signals
  [UNHEALTHY] VIX 22.6 (elevated fear - caution)

CRISIS BUYING MODE
STATUS: INACTIVE - Market is healthy
Use 02.1_price_finder.ipynb for normal trading.


## Fundamental Analysis - Quality Filter

In a crisis, we ONLY want the highest quality companies that will survive and thrive

In [3]:
def get_fundamentals(ticker):
    """Get fundamental data with focus on survival metrics"""
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        # Growth metrics
        revenue_growth = info.get('revenueGrowth', 0) * 100 if info.get('revenueGrowth') else None
        quarterly_revenue_growth = info.get('quarterlyRevenueGrowth', 0) * 100 if info.get('quarterlyRevenueGrowth') else None
        
        # Profitability
        gross_margin = info.get('grossMargins', 0) * 100 if info.get('grossMargins') else None
        operating_margin = info.get('operatingMargins', 0) * 100 if info.get('operatingMargins') else None
        net_margin = info.get('profitMargins', 0) * 100 if info.get('profitMargins') else None
        
        # Financial health (CRITICAL in crisis)
        current_ratio = info.get('currentRatio', None)  # > 1.5 is good
        debt_to_equity = info.get('debtToEquity', None)  # < 50 is good
        free_cash_flow = info.get('freeCashflow', None)
        
        # Valuation
        pe_ratio = info.get('trailingPE', None)
        peg_ratio = info.get('pegRatio', None)
        price_to_book = info.get('priceToBook', None)
        
        return {
            'revenue_growth_yoy': revenue_growth,
            'revenue_growth_qoq': quarterly_revenue_growth,
            'gross_margin': gross_margin,
            'operating_margin': operating_margin,
            'net_margin': net_margin,
            'current_ratio': current_ratio,
            'debt_to_equity': debt_to_equity,
            'free_cash_flow': free_cash_flow,
            'pe_ratio': pe_ratio,
            'peg_ratio': peg_ratio,
            'price_to_book': price_to_book,
            'market_cap': info.get('marketCap', 0)
        }
    except:
        return None


def calculate_quality_score(fundamentals):
    """Score based on ability to SURVIVE crisis (0-100)"""
    if not fundamentals:
        return 0, ["No fundamental data"], []
    
    score = 0
    strengths = []
    warnings = []
    
    # Financial Health (0-40 points) - MOST IMPORTANT in crisis
    current_ratio = fundamentals.get('current_ratio')
    if current_ratio:
        if current_ratio > 2.0:
            score += 20
            strengths.append(f"Excellent liquidity (CR {current_ratio:.1f})")
        elif current_ratio > 1.5:
            score += 15
            strengths.append(f"Good liquidity (CR {current_ratio:.1f})")
        elif current_ratio > 1.0:
            score += 8
            strengths.append(f"Adequate liquidity (CR {current_ratio:.1f})")
        else:
            warnings.append(f"WARNING: Low liquidity (CR {current_ratio:.1f})")
    
    debt_to_equity = fundamentals.get('debt_to_equity')
    if debt_to_equity is not None:
        if debt_to_equity < 30:
            score += 20
            strengths.append(f"Low debt (D/E {debt_to_equity:.0f})")
        elif debt_to_equity < 50:
            score += 15
            strengths.append(f"Manageable debt (D/E {debt_to_equity:.0f})")
        elif debt_to_equity < 100:
            score += 8
            warnings.append(f"Moderate debt (D/E {debt_to_equity:.0f})")
        else:
            warnings.append(f"WARNING: High debt (D/E {debt_to_equity:.0f})")
    
    # Profitability (0-30 points)
    net_margin = fundamentals.get('net_margin')
    if net_margin:
        if net_margin > 20:
            score += 15
            strengths.append(f"Highly profitable (NM {net_margin:.1f}%)")
        elif net_margin > 10:
            score += 10
            strengths.append(f"Profitable (NM {net_margin:.1f}%)")
        elif net_margin > 0:
            score += 5
            strengths.append(f"Marginally profitable (NM {net_margin:.1f}%)")
        else:
            warnings.append(f"WARNING: Unprofitable (NM {net_margin:.1f}%)")
    
    gross_margin = fundamentals.get('gross_margin')
    if gross_margin and gross_margin > 40:
        score += 15
        strengths.append(f"Strong margins (GM {gross_margin:.1f}%)")
    
    # Growth (0-30 points) - Less important in crisis
    rev_growth = fundamentals.get('revenue_growth_yoy')
    if rev_growth:
        if rev_growth > 20:
            score += 20
            strengths.append(f"Strong growth ({rev_growth:.1f}% YoY)")
        elif rev_growth > 10:
            score += 15
            strengths.append(f"Good growth ({rev_growth:.1f}% YoY)")
        elif rev_growth > 0:
            score += 10
            strengths.append(f"Growing ({rev_growth:.1f}% YoY)")
        else:
            warnings.append(f"Revenue declining ({rev_growth:.1f}% YoY)")
    
    return score, strengths, warnings


def calculate_discount(fundamentals, current_price):
    """Calculate how much stock is discounted from 'fair value'"""
    
    # Simple valuation: Compare P/E to historical average
    pe = fundamentals.get('pe_ratio')
    pb = fundamentals.get('price_to_book')
    peg = fundamentals.get('peg_ratio')
    
    discount_signals = []
    
    if pe and pe < 15:
        discount_signals.append(f"Low P/E {pe:.1f} (cheap)")
    elif pe and pe > 50:
        discount_signals.append(f"High P/E {pe:.1f} (still expensive)")
    
    if pb and pb < 2:
        discount_signals.append(f"Low P/B {pb:.1f} (value)")
    
    if peg and peg < 1:
        discount_signals.append(f"PEG {peg:.1f} (undervalued)")
    
    return discount_signals

## Crisis Analysis Function

In [4]:
def analyze_for_crisis(ticker, fetcher):
    """Analyze stock for crisis buying opportunity"""
    print(f"\nAnalyzing {ticker}...")
    print("="*60)
    
    # Get fundamental data
    fundamentals = get_fundamentals(ticker)
    quality_score, strengths, warnings = calculate_quality_score(fundamentals)
    
    # Get technical data
    df = fetcher.fetch(ticker, period='1y')
    if df is None or len(df) < 60:
        print("Insufficient data")
        return None
    
    df['SMA_50'] = sma(df['Close'], 50)
    df['SMA_200'] = sma(df['Close'], 200)
    df['RSI'] = rsi(df['Close'], 14)
    df['ATR'] = atr(df['High'], df['Low'], df['Close'], 14)
    
    df = df.dropna()
    current = df.iloc[-1]
    current_price = current['Close']
    
    # Calculate drawdown from recent high
    high_52w = df['High'].tail(252).max()
    drawdown_pct = ((current_price - high_52w) / high_52w) * 100
    
    # Display results
    print(f"\nCURRENT PRICE: ${current_price:.2f}")
    print(f"52-Week High: ${high_52w:.2f}")
    print(f"DRAWDOWN: {drawdown_pct:.1f}% from highs")
    
    print(f"\nQUALITY SCORE: {quality_score}/100")
    if strengths:
        print("Strengths:")
        for s in strengths:
            print(f"  + {s}")
    
    if warnings:
        print("Warnings:")
        for w in warnings:
            print(f"  - {w}")
    
    # Valuation
    discount_signals = calculate_discount(fundamentals, current_price)
    if discount_signals:
        print("\nValuation:")
        for d in discount_signals:
            print(f"  {d}")
    
    print(f"\nTECHNICAL STATUS:")
    print(f"  RSI: {current['RSI']:.0f} ({'OVERSOLD' if current['RSI'] < 30 else 'EXTREME OVERSOLD' if current['RSI'] < 20 else 'Normal'})")
    print(f"  50-day MA: ${current['SMA_50']:.2f}")
    print(f"  200-day MA: ${current['SMA_200']:.2f}")
    
    # Crisis buying criteria
    # 1. Quality score > 60 (can survive)
    # 2. Down > 20% from highs (panic selling)
    # 3. RSI < 40 (oversold)
    
    can_survive = quality_score >= 60
    is_discounted = drawdown_pct < -20
    is_oversold = current['RSI'] < 40
    
    if can_survive and is_discounted and is_oversold:
        print(f"\nRECOMMENDATION: STRONG BUY (Crisis Opportunity)")
        print(f"Reason: Quality company at panic prices")
        
        # Calculate position sizing (dollar-cost average)
        entry_1 = current_price
        entry_2 = current_price * 0.90  # 10% lower
        entry_3 = current_price * 0.80  # 20% lower
        
        print(f"\nDOLLAR-COST AVERAGE PLAN:")
        print(f"  Entry 1: ${entry_1:.2f} (33% of capital) - BUY NOW")
        print(f"  Entry 2: ${entry_2:.2f} (33% of capital) - If drops 10%")
        print(f"  Entry 3: ${entry_3:.2f} (34% of capital) - If drops 20%")
        print(f"\nAverage entry: ${(entry_1 + entry_2 + entry_3) / 3:.2f}")
        
        recovery_target = high_52w
        potential_gain = ((recovery_target - current_price) / current_price) * 100
        
        print(f"\nRECOVERY POTENTIAL:")
        print(f"  Target: ${recovery_target:.2f} (52-week high)")
        print(f"  Upside: +{potential_gain:.1f}%")
        
        return {
            'ticker': ticker,
            'action': 'BUY',
            'quality_score': quality_score,
            'current_price': current_price,
            'drawdown_pct': drawdown_pct,
            'entry_1': entry_1,
            'entry_2': entry_2,
            'entry_3': entry_3,
            'upside_pct': potential_gain
        }
    
    elif can_survive and is_discounted:
        print(f"\nRECOMMENDATION: WATCH")
        print(f"Reason: Quality company but not oversold enough yet (RSI {current['RSI']:.0f})")
        
        alert_price = current_price * 0.95
        print(f"\nSET ALERT: ${alert_price:.2f} (5% lower)")
        
        return {
            'ticker': ticker,
            'action': 'WATCH',
            'quality_score': quality_score,
            'current_price': current_price,
            'alert_price': alert_price,
            'drawdown_pct': drawdown_pct
        }
    
    elif can_survive:
        print(f"\nRECOMMENDATION: WAIT")
        print(f"Reason: Quality company but not discounted enough ({drawdown_pct:.1f}% from high)")
        
        return {
            'ticker': ticker,
            'action': 'WAIT',
            'quality_score': quality_score,
            'current_price': current_price,
            'drawdown_pct': drawdown_pct
        }
    
    else:
        print(f"\nRECOMMENDATION: AVOID")
        print(f"Reason: Quality score too low ({quality_score}/100) - survival risk")
        return None

## Scan Blue-Chip Stocks for Crisis Opportunities

Focus on companies that will definitely survive and recover

In [5]:
fetcher = StockDataFetcher()

# Blue-chip + quality growth stocks
crisis_watchlist = [
    # Mega-cap tech
    'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'TSLA',
    
    # Cloud/SaaS leaders
    'CRM', 'NOW', 'SNOW', 'DDOG', 'NET', 'MDB', 'CRWD',
    
    # Fintech
    'V', 'MA', 'PYPL', 'COIN', 'HOOD', 'SOFI',
    
    # E-commerce
    'SHOP', 'MELI', 'DASH',
    
    # Semiconductors
    'TSM', 'AMAT', 'LRCX', 'MRVL',
    
    # Healthcare
    'UNH', 'HIMS',
    
    # Innovation
    'PLTR', 'ADBE'
]

if not CRISIS_MODE_ACTIVE:
    print("\n" + "="*60)
    print("WARNING: Crisis mode is NOT active")
    print("Running analysis for educational purposes only.")
    print("DO NOT deploy capital until crisis conditions exist.")
    print("="*60 + "\n")

results = []
for ticker in crisis_watchlist:
    result = analyze_for_crisis(ticker, fetcher)
    if result:
        results.append(result)

# Summary
print("\n" + "="*60)
print("CRISIS BUYING SUMMARY")
print("="*60)

buy_now = [r for r in results if r['action'] == 'BUY']
watch = [r for r in results if r['action'] == 'WATCH']
wait = [r for r in results if r['action'] == 'WAIT']

if buy_now:
    print(f"\nSTRONG BUY - Crisis Opportunities ({len(buy_now)} stocks):")
    print("Quality companies at panic prices - Start DCA now\n")
    buy_sorted = sorted(buy_now, key=lambda x: x['upside_pct'], reverse=True)
    for r in buy_sorted:
        print(f"  {r['ticker']}: ${r['current_price']:.2f} (Down {r['drawdown_pct']:.1f}%) - Upside: +{r['upside_pct']:.1f}% [Quality: {r['quality_score']}]")
        print(f"    DCA: ${r['entry_1']:.2f} -> ${r['entry_2']:.2f} -> ${r['entry_3']:.2f}")

if watch:
    print(f"\nWATCH - Getting close ({len(watch)} stocks):")
    for r in watch:
        print(f"  {r['ticker']}: ${r['current_price']:.2f} -> Alert: ${r['alert_price']:.2f} [Quality: {r['quality_score']}]")

if wait:
    print(f"\nWAIT - Not discounted enough ({len(wait)} stocks):")
    for r in wait:
        print(f"  {r['ticker']}: ${r['current_price']:.2f} (Down {r['drawdown_pct']:.1f}%) [Quality: {r['quality_score']}]")

print("\n" + "="*60)
print("CRISIS BUYING RULES:")
print("="*60)
print("1. ONLY buy during bear markets/corrections")
print("2. ONLY buy quality companies (score > 60)")
print("3. ALWAYS dollar-cost average (3 entries)")
print("4. Hold for 2-5 years (recovery + growth)")
print("5. Size positions: 5-10% of portfolio max")
print("="*60)


Running analysis for educational purposes only.
DO NOT deploy capital until crisis conditions exist.


Analyzing AAPL...
Fetching AAPL data...
Got 250 days of data

CURRENT PRICE: $270.84
52-Week High: $277.05
DRAWDOWN: -2.2% from highs

QUALITY SCORE: 40/100
Strengths:
  + Highly profitable (NM 26.9%)
  + Strong margins (GM 46.9%)
  + Growing (7.9% YoY)
Warnings:
  - WARNING: Low liquidity (CR 0.9)
  - WARNING: High debt (D/E 152)

TECHNICAL STATUS:
  RSI: 66 (Normal)
  50-day MA: $255.00
  200-day MA: $224.71

RECOMMENDATION: AVOID
Reason: Quality score too low (40/100) - survival risk

Analyzing MSFT...
Fetching MSFT data...
Got 250 days of data

CURRENT PRICE: $502.05
52-Week High: $553.72
DRAWDOWN: -9.3% from highs

QUALITY SCORE: 68/100
Strengths:
  + Adequate liquidity (CR 1.4)
  + Manageable debt (D/E 33)
  + Highly profitable (NM 35.7%)
  + Strong margins (GM 68.8%)
  + Good growth (18.4% YoY)

TECHNICAL STATUS:
  RSI: 38 (Normal)
  50-day MA: $513.93
  200-day MA: $463.39

R

## Single Stock Deep Dive

In [ ]:
# Analyze specific stock in detail
ticker_to_analyze = 'NVDA'
result = analyze_for_crisis(ticker_to_analyze, fetcher)